In [529]:
import pandas as pd
import numpy as np
import re as re

In [530]:
!pip3 install chardet

In [531]:
pwd

'/Users/jm/Desktop/ironhack/curso/shark-project'

In [532]:
import chardet

with open('./Input/GSAF5.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large


df = pd.read_csv('./Input/GSAF5.csv', encoding=result['encoding'])
print(df.shape)

(5992, 24)


In [533]:
#Busco los nulos existentes en las columnas
null_cols = df.isnull().sum()
null_cols

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
pdf                          0
href formula                 1
href                         3
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

In [534]:
#Elimino unnamed22 y unnamed 23 porque no continen datos
drop_cols = list(null_cols[null_cols > 5000].index)
df = df.drop(drop_cols, axis=1)
print(df.shape)

(5992, 22)


In [535]:
#Elimino CaseNumber.1 y CaseNumber.2 porque son un duplicado de la primera columna CaseNumber
df = df.drop(columns=['Case Number.1', 'Case Number.2'])
print(df.shape)

(5992, 20)


In [536]:
''' 
Sale False porque no son exactamente iguales ya que al ver cuantos nulos hay en cada uno en una salen más que en otra,
pero observando los datos las URL son iguales, por lo que borramos la que más nulos tiene.
'''
df['href formula'].equals(df['href'])

df = df.drop(columns=['href'])
print(df.shape)

(5992, 19)


In [537]:
#La columna pdf se refiere al pdf que surje de la URL por lo que no nos da información extra.
df = df.drop(columns=['pdf'])
print(df.shape)

(5992, 18)


In [538]:
df["Fatal (Y/N)"].value_counts().sort_index()

 N            8
#VALUE!       1
F             1
N          4315
N             1
UNKNOWN      94
Y          1552
n             1
Name: Fatal (Y/N), dtype: int64

In [539]:
#Unifico valores en la columna Fatal para saber cuantos han sido mortales y cuales no
df = df.rename(columns={"Fatal (Y/N)":"Fatal"})
df.Fatal = df.Fatal.replace({"N": "N", "n": "N", "#VALUE!": "N", "F":"Y", "UNKNOWN":"N", " N": "N", "N ": "N", "Y": "Y"})


In [540]:
df["Fatal"].value_counts().sort_index()

N    4420
Y    1553
Name: Fatal, dtype: int64

In [541]:
df

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Investigator or Source,href formula,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5987,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",http://sharkattackfile.net/spreadsheets/pdf_di...,6
5988,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",http://sharkattackfile.net/spreadsheets/pdf_di...,5
5989,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...,4
5990,ND.0002,1883-1889,0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",http://sharkattackfile.net/spreadsheets/pdf_di...,3


In [542]:
df["Year"].value_counts().sort_index()


0       124
5         1
77        1
500       1
1543      1
       ... 
2012    117
2013    122
2014    125
2015    139
2016    103
Name: Year, Length: 232, dtype: int64

In [543]:
#Elimino las filas con un año inferior a 1850 porque la gente de esa época bebía mucho y no hacia bien los registros :)
df = df.drop(df[df['Year']<1850].index)
df

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Investigator or Source,href formula,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,5989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5751,1852.00.00,1852,1852,Unprovoked,USA,South Carolina,"Mount Pleasant, Charleston County","Vessel capsized, wading ashore carrying an oar",Charles Chambers,M,NaN,"FATAL, body was not recovered",Y,NaN,NaN,"W. H. Gregg, p. 21",http://sharkattackfile.net/spreadsheets/pdf_di...,242
5752,1851.06.19.R,Reported 19-Jun-1851,1851,Unprovoked,MEXICO,Oaxaca,NaN,Bathing,John Gray & another member of the Tehuantepec ...,M,NaN,FATAL,Y,NaN,NaN,"Burlington Hawk-Eye, 6/19/1851",http://sharkattackfile.net/spreadsheets/pdf_di...,241
5753,1851.03.08.R,Mar-1851,1851,Unprovoked,USA,Hawaii,Honolulu Harbor,Swimming,James Kinney,M,NaN,FATAL,Y,NaN,NaN,"The Friend (Honolulu), 3/8/1851",http://sharkattackfile.net/spreadsheets/pdf_di...,240
5754,1851.00.00,1851,1851,Unprovoked,USA,California,"San Francisco Bay (or San Leandro Bay), near c...",Hard hat diving,William Cortigan,M,NaN,3 toes severed,N,NaN,18' shark,"The Perry Chief, 10/16/1875",http://sharkattackfile.net/spreadsheets/pdf_di...,239


In [544]:
df["Year"].value_counts().sort_index()

1850      1
1851      3
1852     10
1853      7
1854      1
       ... 
2012    117
2013    122
2014    125
2015    139
2016    103
Name: Year, Length: 166, dtype: int64

In [545]:
#Ataques últimos 50 años de 1800 son entre 1-20 al año por lo que no es muy representativo
x = df[(df["Year"] <1900)]
x["Year"].value_counts().sort_index()
df = df.drop(df[df['Year']<1900].index)
y = df[(df["Year"] <1950)]
y["Year"].value_counts().sort_index()
'''Ataques primeros 50 años de 1900 son como mucho 40 al año por lo que no es muy representativo
Mi hipotesis comenzará a partir de los años 50
'''
df = df.drop(df[df['Year']<1950].index)
z = df[(df["Year"] <2000)]
z["Year"].value_counts().sort_index()


1950    43
1951    31
1952    29
1953    36
1954    42
1955    43
1956    51
1957    41
1958    54
1959    93
1960    93
1961    78
1962    86
1963    61
1964    66
1965    51
1966    58
1967    48
1968    46
1969    30
1970    42
1971    28
1972    35
1973    27
1974    38
1975    49
1976    39
1977    26
1978    25
1979    25
1980    35
1981    49
1982    40
1983    50
1984    41
1985    37
1986    39
1987    35
1988    55
1989    53
1990    38
1991    38
1992    56
1993    56
1994    56
1995    76
1996    61
1997    57
1998    65
1999    65
Name: Year, dtype: int64

In [546]:
'''
Al eliminar las últimas filas la columna original order no tiene sentido,
también elimino area y location porque mi hipotesis se va a centrar en paises
y el Investigator or Source y name porque no da un dato que me interese
'''
df = df.drop(columns=['Area', 'Location', 'original order', 'Investigator or Source', 'Name'])

In [547]:
df

,Case Number,Date,Year,Type,Country,Activity,Sex,Age,Injury,Fatal,Time,Species,href formula
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Surfing,M,16,Minor injury to thigh,N,13h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Surfing,M,36,Lacerations to hands,N,11h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Surfing,M,43,Lacerations to lower leg,N,10h43,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Surfing,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Surfing,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4266,1950.00.00.e,Summer 1950,1950,Unprovoked,GREECE,Swimming,NaN,NaN,FATAL,Y,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
4267,1950.00.00.d,1950,1950,Unprovoked,SINGAPORE,Diving for coins,M,NaN,FATAL,Y,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
4268,1950.00.00.c,1950,1950,Unprovoked,NEW CALEDONIA,"Spearfishing, but walking carrying fish on end...",M,NaN,"Shark jumped from sea, taking fish & his right...",N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
4269,1950.00.00.b,Ca. 1950,1950,Unprovoked,NEW CALEDONIA,"Helmet diving, collecting trochus shell",M,NaN,"Arm bitten, surgically amputated",N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...


In [548]:
#Renombro los paises mal escritos
df["Country"].value_counts()
df.Country = df.Country.replace({" PHILIPPINES": "PHILIPPINES", " TONGA": "TONGA"})

In [549]:
#Agrupo boating y boat
df.Type = df.Type.replace({"Boating": "Boat"})

In [550]:
#Quito el espacio del nombre de la columna y unifico valores
df = df.rename(columns={"Sex ":"Sex"})
df["Sex"].value_counts()
df.Sex = df.Sex.replace({"M ": "M"})


In [551]:
df["Sex"].value_counts()
print(df.shape)

(4271, 13)


In [552]:
#Elimino los datos que no me interesan ya que quiero saber por ataques a hombres o a mujeres
df = df.drop(df[(df['Sex']==".")].index)
df = df.drop(df[(df['Sex']=="lli")].index)
df = df.drop(df[(df['Sex']=="N")].index)
print(df.shape)

(4270, 13)


In [553]:
df["Sex"].value_counts()

M    3385
F     491
Name: Sex, dtype: int64

In [554]:
df

,Case Number,Date,Year,Type,Country,Activity,Sex,Age,Injury,Fatal,Time,Species,href formula
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Surfing,M,16,Minor injury to thigh,N,13h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Surfing,M,36,Lacerations to hands,N,11h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Surfing,M,43,Lacerations to lower leg,N,10h43,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Surfing,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Surfing,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4266,1950.00.00.e,Summer 1950,1950,Unprovoked,GREECE,Swimming,NaN,NaN,FATAL,Y,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
4267,1950.00.00.d,1950,1950,Unprovoked,SINGAPORE,Diving for coins,M,NaN,FATAL,Y,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
4268,1950.00.00.c,1950,1950,Unprovoked,NEW CALEDONIA,"Spearfishing, but walking carrying fish on end...",M,NaN,"Shark jumped from sea, taking fish & his right...",N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
4269,1950.00.00.b,Ca. 1950,1950,Unprovoked,NEW CALEDONIA,"Helmet diving, collecting trochus shell",M,NaN,"Arm bitten, surgically amputated",N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...


In [555]:
df["Age"].value_counts()

19                118
20                114
17                113
15                113
16                113
                 ... 
21, 34,24 & 35      1
86                  1
21 & ?              1
mid-30s             1
50s                 1
Name: Age, Length: 136, dtype: int64

In [556]:
#Limpio la columna edad y lo paso a enteros
df['Age'] = df['Age'].str.replace('[a-z]', ' ')
df['Age'] = df['Age'].str.replace('[A-Z]', ' ')
df['Age'] = df['Age'].str.replace('\?', '')
df['Age'] = df['Age'].str.replace('.', ' ')
df['Age'] = df['Age'].str.replace('&', ' ')
df['Age'] = df['Age'].str.replace('-', ' ')
df['Age'] = df['Age'].str.replace('>', ' ')
df['Age'] = df['Age'].str.replace('\(', '')
df['Age'] = df['Age'].str.replace('\)', '')
df['Age'] = df['Age'].str.replace('½', '')
df['Age'] = df['Age'].str.replace('\xa0 ', '')
df['Age'] = df['Age'].str.replace('\s+', '')
df['Age'] = df['Age'].str.replace('""', '')
df = df.drop(df[(df['Age']=="")].index)
df.shape

(4248, 13)

In [557]:
df = df[df['Age'].str.len().lt(3)]
print(set(df['Age']))
df['Age'].value_counts()
df.dtypes
df.shape

{'42', '7', '27', '9', '51', '69', '24', '19', '41', '72', '73', '12', '25', '36', '33', '56', '81', '11', '40', '87', '47', '43', '17', '65', '66', '28', '75', '38', '45', '55', '16', '21', '23', '10', '26', '86', '46', '57', '60', '13', '1', '32', '30', '84', '22', '77', '59', '67', '74', '58', '49', '52', '78', '50', '54', '62', '6', '18', '31', '29', '63', '5', '8', '34', '35', '64', '71', '68', '70', '61', '14', '20', '48', '44', '37', '39', '15', '53', '3'}


(2823, 13)

In [558]:
df["Age"] = df["Age"].astype(int)
df.shape

(2823, 13)

In [559]:
df.dtypes

Case Number     object
Date            object
Year             int64
Type            object
Country         object
Activity        object
Sex             object
Age              int64
Injury          object
Fatal           object
Time            object
Species         object
href formula    object
dtype: object

In [560]:
df['count'] = df.groupby('Country')['Country'].transform('count')
df = df.rename(columns={'count':'Attacks'})

In [561]:
df

,Case Number,Date,Year,Type,Country,Activity,Sex,Age,Injury,Fatal,Time,Species,href formula,Attacks
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Surfing,M,16,Minor injury to thigh,N,13h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,1349.0
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Surfing,M,36,Lacerations to hands,N,11h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,1349.0
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Surfing,M,43,Lacerations to lower leg,N,10h43,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,1349.0
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Wading,M,60,Minor injury to arm,N,15h15,3' to 4' shark,http://sharkattackfile.net/spreadsheets/pdf_di...,1349.0
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Swimming,F,51,Severe lacerations to shoulder & forearm,N,14h30,"Tiger shark, 10?",http://sharkattackfile.net/spreadsheets/pdf_di...,1349.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4247,1950.05.24,24-May-50,1950,Unprovoked,SOUTH AFRICA,Treading water,M,27,Left foot bitten,N,10h30,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,305.0
4249,1950.04.09.b,09-Apr-50,1950,Sea Disaster,AUSTRALIA,"Sea Disaster, sinking of the fishing launch M...",M,50,FATAL,Y,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,486.0
4252,1950.03.08,08-Mar-50,1950,Unprovoked,SOUTH AFRICA,Lifesaving drill,M,20,FATAL,Y,18h50,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,305.0
4255,1950.02.11,11-Feb-50,1950,Unprovoked,SOUTH AFRICA,Body surfing,M,14,"FATAL, body not recovered",Y,14h30,"White shark, 3.7 m [12'] according to witnesses",http://sharkattackfile.net/spreadsheets/pdf_di...,305.0


In [562]:
#Por qué no me funciona esto?
#df3 = df.drop(df[(df['Date']!='[0-9]+.[A-Z][a-z]+.[0-9]+')].index)
#df3
#a = str(df['Date'])
#print(re.findall("[0-9]+.[A-Z][a-z]+.[0-9]+", a))

In [566]:
x = df.groupby(['Attacks', 'Country']).max()
x

Age   Case Number                  Date  \
Attacks Country                                                               
1.0     ADMIRALTY ISLANDS            26    1962.10.15             15-Oct-62   
        ANDAMAN / NICOBAR ISLANDAS   32    1958.09.13             13-Sep-58   
        ARGENTINA                    18  1954.01.22.a             22-Jan-54   
        ARUBA                        58    2015.12.19             19-Dec-15   
        AZORES                       49  2009.01.27.R  Reported 27-Jan-2009   
...                                 ...           ...                   ...   
56.0    BRAZIL                       43    2016.03.02  Reported 26-Jan-2009   
59.0    BAHAMAS                      77    2016.08.07                Oct-06   
305.0   SOUTH AFRICA                 77    2016.06.23                Sep-75   
486.0   AUSTRALIA                    84    2016.07.26  Reported 29-Oct-1989   
1349.0  USA                          87  2016.09.18.c        Summer of 1996   

                                          Type  Year  
Attacks Country                                       
1.0     ADMIRALTY ISLANDS           Unprovoked  1962  
        ANDAMAN / NICOBAR ISLANDAS  Unprovoked  1958  
        ARGENTINA                   Unprovoked  1954  
        ARUBA                       Unprovoked  2015  
        AZORES                        Provoked  2009  
...                                        ...   ...  
56.0    BRAZIL                      Unprovoked  2016  
59.0    BAHAMAS                     Unprovoked  2016  
305.0   SOUTH AFRICA                Unprovoked  2016  
486.0   AUSTRALIA                   Unprovoked  2016  
1349.0  USA                         Unprovoked  2016  

[113 rows x 5 columns]

In [564]:
df['Type'].value_counts()


Unprovoked      2354
Provoked         234
Invalid          187
Sea Disaster      29
Boat              19
Name: Type, dtype: int64